<a href="https://colab.research.google.com/github/rajkamaljangra/chatbot_from_corpus/blob/main/chatbot_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import the required modules**

In [1]:
import numpy as numpy
import nltk
import string
import random

**Importing the corpus**

In [ ]:
f=open('chatbot.txt','r', errors='ignore')
raw_doc = f.read()
raw_doc

In [17]:
raw_doc=raw_doc.lower()
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
sent_tokens = nltk.sent_tokenize(raw_doc)
word_tokens = nltk.word_tokenize(raw_doc)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Test preprocessing**

In [18]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]

remove_punch_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punch_dict)))


**Define Greetings**

In [22]:
GREET_INPUTS = ('hello','hi','greetings','how are you','hey',)
GREET_RESPONSES = ['hi','hey', 'hi there', 'hello', 'i am glad talking to you']
def greet(sentence):
  for word in sentence.split():
    if word.lower() in GREET_INPUTS:
      return random.choice(GREET_RESPONSES)

**Response generation**

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def response(user_response):
  robo1_response =''
  TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words = 'english')
  tfidf = TfidfVec.fit_transform(sent_tokens)
  vals = cosine_similarity(tfidf[-1],tfidf)
  idx = vals.argsort()[0][-2]
  flat= vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if req_tfidf==0:
    robo1_response = robo1_response + "I am sorry, I donot understand you"
    return robo1_response
  else:
    robo1_response = robo1_response + sent_tokens[idx]
    return robo1_response

**Define chat start and end conditions**

In [ ]:
flag = True
print("BOT: My name is RKJBot, ask me any question. To exit at any time, just type Bye!")
while(flag==True):
  user_response=input()
  user_response=user_response.lower()
  if(user_response!='bye'):
    if (user_response=='thanks' or user_response=='thank you'):
      flag=False
      print("BOT: You are welcome..")
    else:
      if (greet (user_response)!=None):
        print("BOT: "+greet(user_response))
      else:
        sent_tokens.append(user_response)
        word_tokens=word_tokens+nltk.word_tokenize(user_response)
        final_words=list(set(word_tokens))
        print("BOT: ",end="")
        print(response(user_response))
        sent_tokens.remove(user_response)
  else:
    flag=False
    print("BOT: Goodbye!")

